```python
# input_data shape
Input: (batch_size, in_channel, width, height)
# conv layer
class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
input: (Batch_size, C_in, H_in, W_in)
output: (Batch_size, C_out, H_out, W_out)

weight(tensor): (out_channels, in_channels,kernel_size)
bias(tensor): (out_channel)
```

In [10]:
from itertools import product

import time
import numpy as np
from sklearn import datasets
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt

from ripser import Rips
from persim import PersistenceImager

import os
import math

import seaborn
import pandas as pd
import torch
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader

import re
import cv2
from PIL import Image

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 5


In [12]:
# dir = os.listdir("./metric")
# data = []
# a = []
# for metric in dir:
#     a = re.findall("\d+\.?\d*", metric)
#     data.append([np.loadtxt("./metric/"+metric), metric])
#     # print(a[-2])
#     print(metric[-2])


In [13]:
class My_dataset(Dataset):
    def __init__(self, train):
        super().__init__()
        # 使用sin函数返回10000个时间序列,如果不自己构造数据，就使用numpy,pandas等读取自己的数据为x即可。
        # 以下数据组织这块既可以放在init方法里，也可以放在getitem方法里
        dir = os.listdir("./metric_trainset_cifar")
        dirtest = os.listdir("./metric_testset_cifar")
        data = []
        a = []
        if train:
            # for metric in dir:
            #     data.append(
            #         [np.loadtxt("./grids_trainset_cifar/"+metric), metric])
            sample = []
            for metric in dir:
                sample.append([np.loadtxt("./metric_trainset_cifar/"+metric), metric])
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    data.append([res, metric])
                    sample = []
            
        else:
            # for metric in dirtest:
            #     data.append(
            #         [np.loadtxt("./grids_testset_cifar/"+metric), metric])
            sample = []
            for metric in dirtest:
                sample.append([np.loadtxt("./metric_testset_cifar/"+metric), metric])
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    data.append([res, metric])
                    sample = []

        self.x = [item[0] for item in data]
        self.y = [math.ceil(int(re.findall("\d+\.?\d*", item[1])[0]) /
                            (int(re.findall("\d+\.?\d*", item[1])[0])+1)) for item in data]
        self.src,  self.trg = [], []
        for i in range(len(data)):
            self.src.append(self.x[i])
            self.trg.append(self.y[i])

    def __getitem__(self, index):
        return self.src[index], self.trg[index]

    def __len__(self):
        return len(self.src)

 # 或者return len(self.trg), src和trg长度一样


In [14]:

data_train = My_dataset(train=True)
data_test = My_dataset(train=False)
data_loader_train = DataLoader(data_train, batch_size=10, shuffle=False)
data_loader_test = DataLoader(data_test, batch_size=10, shuffle=False)
# i_batch的多少根据batch size和def __len__(self)返回的长度确定
# batch_data返回的值根据def __getitem__(self, index)来确定
# 对训练集：(不太清楚enumerate返回什么的时候就多print试试)
for i_batch, batch_data in enumerate(data_loader_train):
    print(i_batch)  # 打印batch编号
    print(batch_data[0])  # 打印该batch里面src
    print(batch_data[1])  # 打印该batch里面trg
# 对测试集：（下面的语句也可以）
for i_batch, (src, trg) in enumerate(data_loader_test):
    print(i_batch)  # 打印batch编号
    print(src)  # 打印该batch里面src的尺寸
    print(trg)  # 打印该batch里面trg的尺寸


0
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.

In [15]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 25, kernel_size=3),
            torch.nn.BatchNorm2d(25),
            torch.nn.ReLU(inplace=True)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(25, 50, kernel_size=3),
            torch.nn.BatchNorm2d(50),
            torch.nn.ReLU(inplace=True)
        )

        self.layer4 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(50 * 30 * 30, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


model = MyNet().to(device)
print(model)


MyNet(
  (layer1): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer4): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=45000, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1024, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [16]:
# define cost/loss & optimizer
# Softmax is internally computed.
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [17]:
total_batch = len(data_loader_train)
model.train()
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader_train:
        X = torch.autograd.Variable(X).to(device).unsqueeze(1).float()
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')


Epoch: 0001 cost = 8.333319664
Epoch: 0002 cost = 12.684720039
Epoch: 0003 cost = 0.288190633
Epoch: 0004 cost = 3.563498497
Epoch: 0005 cost = 0.338373125
Epoch: 0006 cost = 1.017654181
Epoch: 0007 cost = 0.700443864
Epoch: 0008 cost = 0.290733248
Epoch: 0009 cost = 0.595973730
Epoch: 0010 cost = 0.123072848
Epoch: 0011 cost = 0.277883649
Epoch: 0012 cost = 0.094375141
Epoch: 0013 cost = 0.102002814
Epoch: 0014 cost = 0.061731696
Epoch: 0015 cost = 0.030528700
Epoch: 0016 cost = 0.036752582
Epoch: 0017 cost = 0.028096948
Epoch: 0018 cost = 0.020324528
Epoch: 0019 cost = 0.018263854
Epoch: 0020 cost = 0.017441429
Learning finished


In [18]:
# Test the model using test sets
# model.load_state_dict(torch.load(".\merge_save\merge_attack0normal10_05-07--21-10-32.pth")["state_dict"])
# model.load_state_dict(torch.load(r".\myfed_normal_save\model18-24-02.pth")["state_dict"])
model.eval()
with torch.no_grad():
    for X_test, Y_test in data_loader_test:
        X_test = X_test.to(device).unsqueeze(1).float()
        Y_test = Y_test.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    # r = random.randint(0, len(mnist_test) - 1)
    # X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    # Y_single_data = mnist_test.targets[r:r + 1].to(device)

    # print('Label: ', Y_single_data.item())
    # single_prediction = model(X_single_data)
    # print('Prediction: ', torch.argmax(single_prediction, 1).item())


Accuracy: 0.6000000238418579
